## FD ANN SINGLE DISTANCE (Hb2)

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/MCX_data'
sys.path.append(folder_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import sys
import pickle
from tqdm import tqdm
! pip install pmcx
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import StandardScaler

### Read the saved CSV

In [ ]:
import pandas as pd
import glob
import numpy as np
import sys
! pip install pmcx
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/MCX_data'
sys.path.append(folder_path)
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv_path = '/content/drive/MyDrive/MCX_data/stage2_train.csv'
final_array = np.loadtxt(csv_path, delimiter=',')
print(final_array.shape)
X_train = final_array
#
csv_path_test = '/content/drive/MyDrive/MCX_data/stage2_test.csv'
final_array = np.loadtxt(csv_path_test, delimiter=',')
print(final_array.shape)
X_test = final_array

(10000, 192)
(1000, 192)


### GT

In [ ]:
GT_folder_train = '/content/drive/MyDrive/MCX_data/stage2_csv/'
GT_folder_test = '/content/drive/MyDrive/MCX_data/test_csv/'

In [ ]:
csv_files_train = glob.glob(os.path.join(GT_folder_train, '*.csv'))
GT_all_train = pd.concat([pd.read_csv(f) for f in csv_files_train], ignore_index=True)
csv_files_test = glob.glob(os.path.join(GT_folder_test, '*.csv'))
GT_all_test = pd.concat([pd.read_csv(f) for f in csv_files_test], ignore_index=True)

In [ ]:
sorted_ids = [i+1 for i in range(10000)]
# Ensure ID column is integer
GT_all_train['ID'] = GT_all_train['ID'].astype(int)

# Filter and sort GT_all to match the sorted_ids order
GT_filtered = GT_all_train[GT_all_train['ID'].isin(sorted_ids)].copy()
GT_filtered = GT_filtered.set_index('ID').loc[sorted_ids].reset_index()

# Extract HHB2 as Y
Y = GT_filtered['HHB2'].values
y_train = Y.reshape(-1, 1)

In [ ]:
sorted_ids = [i+1 for i in range(1000)]
# Ensure ID column is integer
GT_all_test['ID'] = GT_all_test['ID'].astype(int)

# Filter and sort GT_all to match the sorted_ids order
GT_filtered = GT_all_test[GT_all_test['ID'].isin(sorted_ids)].copy()
GT_filtered = GT_filtered.set_index('ID').loc[sorted_ids].reset_index()

# Extract HHB2 as Y
Y = GT_filtered['HHB2'].values
y_test = Y.reshape(-1, 1)

In [ ]:
# fit only on training data
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train)

# apply (same μ,σ) everywhere
X_train = scaler_X.transform(X_train)
X_test  = scaler_X.transform(X_test)
Y_train = scaler_y.transform(y_train)
Y_test = scaler_y.transform(y_test)

In [ ]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(10000, 192) (1000, 192) (10000, 1) (1000, 1)


### Seperate

In [ ]:
import numpy as np
import itertools
from typing import Dict, List, Tuple

def split_24x8_distance_pairs(
    X_train: np.ndarray,         # (N_train, 192)
    X_test:  np.ndarray          # (N_test , 192)
) -> Dict[int, List[np.ndarray]]:
    """
    Build a dictionary for *all two-distance* combinations.
    Each entry contains a 48-feature slice (24 + 24).

    Key convention
    --------------
    12 → distances 1 & 2      (cols 0–23  + 24–47)
    13 → distances 1 & 3      (cols 0–23  + 48–71)
    …
    78 → distances 7 & 8      (cols 144–167 + 168–191)

    Returns
    -------
    pair_dict[key] = [ X_train_pair , X_test_pair ]
        where each array has shape (N, 48)
        and len(pair_dict) == 28  (C(8,2))
    """
    if X_train.shape[1] != 192 or X_test.shape[1] != 192:
        raise ValueError("Expect 192 columns = 8 distances × 24 features.")

    # first slice each single-distance block (N, 24)
    single: Dict[int, Tuple[np.ndarray, np.ndarray]] = {}
    for d in range(1, 9):
        s, e = (d - 1) * 24, d * 24
        single[d] = (X_train[:, s:e], X_test[:, s:e])

    # now build every 2-distance combination
    pair_dict: Dict[int, List[np.ndarray]] = {}
    for a, b in itertools.combinations(range(1, 9), 2):
        key = int(f"{a}{b}")                      # e.g. 12, 37, 78
        Xtr_pair = np.concatenate([single[a][0], single[b][0]], axis=1)  # (N, 48)
        Xte_pair = np.concatenate([single[a][1], single[b][1]], axis=1)  # (N, 48)
        pair_dict[key] = [Xtr_pair, Xte_pair]

    return pair_dict

distance_data = split_24x8_distance_pairs(X_train, X_test)

In [ ]:
len(distance_data)

28

### ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.losses import Huber
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from pathlib import Path
import joblib, tensorflow as tf
WEIGHT_DIR = Path("/content/drive/MyDrive/MCX_data/weights")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
Path = '/content/drive/MyDrive/MCX_data/distance_2/'
for idx, (k, v) in enumerate(distance_data.items()):
     model = Sequential([
    Input(shape=(48,)),
    Dense(1000, activation='relu'),
    Dropout(0.1),

    Dense(4096, activation='relu'),
    Dropout(0.1),

    Dense(4096, activation='relu'),
    Dropout(0.2),

    Dense(8192, activation='relu'),
    Dropout(0.2),

    Dense(8192*2, activation='relu'),
    Dropout(0.2),

    Dense(8192, activation='relu'),
    Dropout(0.2),

    Dense(4096, activation='relu'),
    Dropout(0.2),

    Dense(2048, activation='relu'),
    Dropout(0.2),

    Dense(1024, activation='relu'),
    Dropout(0.2),

    Dense(512, activation='relu'),
    Dropout(0.1),

    Dense(256, activation='relu'),
    Dropout(0.1),

    Dense(64, activation='relu'),
    Dropout(0.1),

    Dense(32, activation='relu'),
    Dropout(0.0),

    Dense(1)
])
     checkpoint_cb = ModelCheckpoint(
    "FD_hhb2_ANN.keras",        # filepath to save to
    monitor="val_loss",     # quantity to monitor
    mode="min",             # we want to minimize val_loss
    save_best_only=True,    # only save when val_loss decreases
    save_weights_only=False # set True if you only want weights
)
     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss="mse")

     history = model.fit( v[0], Y_train, epochs=200, batch_size=32, validation_split=0.1, verbose=1, callbacks=[checkpoint_cb])
     model = tf.keras.models.load_model("FD_hhb2_ANN.keras")
     y_pred = model.predict(v[1])
     y_pred = np.clip(y_pred, scaler_y.transform(np.array([[15]]))[0][0], scaler_y.transform(np.array([[35]]))[0][0])
     ### rescale
     y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
     y_test = scaler_y.inverse_transform(Y_test.reshape(-1, 1)).flatten()
     mse = mean_squared_error(y_test, y_pred)
     print(f"MSE for distance {k}: {mse}")
     print(f"RMSE for distance {k}: {np.sqrt(mse)}")
     df = pd.DataFrame({
        "y_test" : y_test,
        "y_pred" : y_pred
    })
     df.to_csv(Path + f"distance_{k}_hhb2_FD.csv", index=False)

Epoch 1/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 67s 199ms/step - loss: 1.0034 - val_loss: 1.0002
Epoch 2/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 26s 91ms/step - loss: 0.9812 - val_loss: 0.9996
Epoch 3/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 25s 88ms/step - loss: 1.0008 - val_loss: 0.9987
Epoch 4/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 26s 92ms/step - loss: 0.9887 - val_loss: 0.9984
Epoch 5/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 26s 94ms/step - loss: 1.0210 - val_loss: 0.9908
Epoch 6/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 25s 88ms/step - loss: 0.9958 - val_loss: 0.9776
Epoch 7/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.9641 - val_loss: 1.0426
Epoch 8/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 25s 88ms/step - loss: 0.9736 - val_loss: 0.9204
Epoch 9/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.9271 - val_loss: 0.9230
Epoch 10/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.9323 - val_loss: 0.9297
Epoch 11/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.9207 - val_loss: 0.9299
Epoch 12/200
282/282 ━